In [1]:
from pathlib import Path
from os.path import dirname, realpath
from datetime import datetime
import os
import shutil
from typing import List, Union
from pycaret.anomaly import *
import numpy as np
import pandas as pd
# import LSTM libraries
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import seaborn as sns
sns.set(color_codes=True)
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.layers import Input, Dropout, Dense, LSTM, TimeDistributed, RepeatVector
from keras.models import Model
from keras import regularizers
#import BIRCH libraries
from sklearn.cluster import Birch
from sklearn import preprocessing
from scipy.spatial import distance

In [13]:
SYSTEM = 'sockshop'
# SYSTEM = 'trainticket'
# Define the main directory
TRAINING_DATA_FOLDER = Path(f"../../vuDevOps/data_collection/{SYSTEM}-training-data")
DATA_FOLDER = Path(f"../../vuDevOps/data_collection/{SYSTEM}-data/")

RESULTS_FOLDER = Path(f"../../experiment_results/{SYSTEM}-data")

AD_FOLDER = 'C:\scul\Thesis\Data Analysis\AD'
NORMAL_DATA_PATH = os.path.join(DATA_FOLDER, "normal")
AD_MODELS = ["iforest", "svm", "lof", "knn"]

# SAVED_AD_MODELS_PATH = os.path.join(AD_FOLDER,"anomaly_detection_models")
# LOCAL_AD_MODELS = os.path.join(SAVED_AD_MODELS_PATH,SYSTEM)

In [16]:
def load_metrics(base_dir):
    data = []
    for root, dirs, files in os.walk(base_dir):
        for file in files:
            if file.endswith("metrics.csv"):
                file_path = os.path.join(root, file)
                df = pd.read_csv(file_path)
                # Extract metadata from the file path
                parts = file_path.split(os.sep)
                scenario = parts[5]
                behavior = parts[6]
                if behavior == 'anomalous':
                    service = parts[7]
                    users = parts[8]
                    repetition = parts[9]
                else:
                    service = 'normal'
                    users = parts[7]
                    repetition = parts[8]
                
                df['scenario'] = scenario
                df['behavior'] = behavior
                df['service'] = service
                df['users'] = users
                df['repetition'] = repetition
                data.append(df)
    return pd.concat(data, ignore_index=True)


In [17]:
data = load_metrics(DATA_FOLDER)
data

,time,carts_cpu,carts-db_cpu,catalogue_cpu,catalogue-db_cpu,front-end_cpu,orders_cpu,orders-db_cpu,payment_cpu,queue-master_cpu,...,user_power,user-db_power,scenario,behavior,service,users,repetition,orders_disk,catalogue_disk,queue-master_disk
0,1.720475e+09,0.039099,0.007015,0.003332,0.001180,0.030832,0.004786,0.005776,0.001558,0.004410,...,0.000874,0.000811,scenario_A,anomalous,front-end,100,repetition_1,NaN,NaN,NaN
1,1.720475e+09,0.059431,0.007996,0.005746,0.003404,0.101100,0.004943,0.005963,0.001533,0.004416,...,0.001258,0.000843,scenario_A,anomalous,front-end,100,repetition_1,NaN,NaN,NaN
2,1.720475e+09,0.065530,0.008247,0.008679,0.005220,0.342384,0.005026,0.005949,0.001326,0.004698,...,0.001460,0.001042,scenario_A,anomalous,front-end,100,repetition_1,NaN,NaN,NaN
3,1.720475e+09,0.077802,0.008251,0.012895,0.009201,0.573051,0.005198,0.006108,0.001483,0.004779,...,0.001629,0.001053,scenario_A,anomalous,front-end,100,repetition_1,NaN,NaN,NaN
4,1.720475e+09,0.085144,0.008638,0.014121,0.010104,0.831620,0.005141,0.005942,0.001470,0.004675,...,0.001735,0.000984,scenario_A,anomalous,front-end,100,repetition_1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21875,1.720430e+09,2.828884,1.960483,0.606384,0.356611,1.109236,0.399870,0.004384,0.022462,0.003844,...,0.135037,0.055955,scenario_B,baseline,normal,1000,repetition_9,NaN,NaN,NaN
21876,1.720430e+09,2.830664,1.851019,0.598874,0.356424,1.109167,0.396980,0.004374,0.022409,0.003727,...,0.118310,0.048866,scenario_B,baseline,normal,1000,repetition_9,NaN,NaN,NaN
21877,1.720430e+09,2.843181,1.951623,0.567690,0.358015,1.108863,0.399207,0.004350,0.022597,0.003826,...,0.118387,0.048518,scenario_B,baseline,normal,1000,repetition_9,NaN,NaN,NaN
21878,1.720430e+09,2.695615,1.963901,0.605374,0.357926,1.108358,0.368062,0.004400,0.022720,0.003611,...,0.118045,0.048746,scenario_B,baseline,normal,1000,repetition_9,NaN,NaN,NaN
